<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/nlpInternship/blob/main/POSTaggIndicBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.47.0 datasets seqeval scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=fff4f0da50d401cc1a075c6db5d2d5b828826195e1c72d33e11d00b11dac4d16
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec


In [ ]:
from datasets import load_dataset
dataset = load_dataset("LingoIITGN/COMI-LINGUA", "POS")  # Load POS task subset
train_data = dataset["train"]
test_data = dataset["test"]

In [3]:
dataset['train'][15683] # dataset['train'][i] the annotaion by all the annotators for the the sentence

{'Sentences': 'जैसलमेर - प्रधानमंत्री नरेंद्र मोदी (Narendra Modi) ने राजस्थान के जैसलमेर बॉर्डर पर लोंगेवाला पोस्ट पहुंच कर सेना के जवानों के साथ दिवाली (Diwali) का त्योहार मनाया।',
 'Predicted Tags': "[{'word': 'जैसलमेर', 'entity': 'PROPN'}, {'word': '-', 'entity': 'X'}, {'word': 'प्रधानमंत्री', 'entity': 'PROPN'}, {'word': 'नरेंद्र', 'entity': 'PROPN'}, {'word': 'मोदी', 'entity': 'PROPN'}, {'word': '(', 'entity': 'X'}, {'word': 'Narendra', 'entity': 'PROPN'}, {'word': 'Modi', 'entity': 'PROPN'}, {'word': ')', 'entity': 'X'}, {'word': 'ने', 'entity': 'VERB'}, {'word': 'राजस्थान', 'entity': 'PROPN'}, {'word': 'के', 'entity': 'ADP'}, {'word': 'जैसलमेर', 'entity': 'PROPN'}, {'word': 'बॉर्डर', 'entity': 'NOUN'}, {'word': 'पर', 'entity': 'ADP'}, {'word': 'लोंगेवाला', 'entity': 'ADJ'}, {'word': 'पोस्ट', 'entity': 'NOUN'}, {'word': 'पहुंच', 'entity': 'VERB'}, {'word': 'कर', 'entity': 'VERB'}, {'word': 'सेना', 'entity': 'NOUN'}, {'word': 'के', 'entity': 'ADP'}, {'word': 'जवानों', 'entity': '

In [4]:
import ast  # safely parse string to Python object

def extract_tokens_and_tags(example):
    ann = ast.literal_eval(example['Annotated by: Annotator 1 '])  # safely convert string to list of dicts
    tokens = [entry["word"] for entry in ann]
    tags = [entry["entity"] for entry in ann]
    return {"tokens": tokens, "tags": tags}


In [ ]:
# Convert to tokens and tags
processed_data = dataset.map(extract_tokens_and_tags)

# Keep only necessary columns
processed_data = processed_data.remove_columns([
    "Sentences", "Predicted Tags",
    "Annotated by: Annotator 1 ",
    "Annotated by: Annotator 2",
    "Annotated by: Annotator 3"
])


In [6]:
train_dataset = processed_data['train']
test_dataset = processed_data['test']

In [7]:
processed_data['train'][0]

{'tokens': ['Loan',
  'Apps',
  'की',
  'अब',
  'खैर',
  'नहीं',
  ',',
  'RBI',
  'ने',
  'बता',
  'दिया',
  'किस',
  '-',
  'किस',
  'पर',
  'होगी',
  'कार्रवाई',
  ',',
  'लिस्ट',
  'तैयार'],
 'tags': ['NOUN',
  'NOUN',
  'ADP',
  'ADV',
  'NOUN',
  'PART_NEG',
  'X',
  'PROPN',
  'ADP',
  'VERB',
  'VERB',
  'PRON_WH',
  'X',
  'PRON_WH',
  'ADP',
  'VERB',
  'NOUN',
  'X',
  'NOUN',
  'VERB']}

In [8]:
# Get all unique tags
unique_tags = list(set(tag for row in train_dataset for tag in row['tags']))
tag2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
id2tag = {i: tag for tag, i in tag2id.items()}
NUM_LABELS = len(tag2id)


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "ai4bharat/indic-bert"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_LABELS,
    id2label=id2tag,
    label2id=tag2id
)


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128

    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val = test_dataset.map(tokenize_and_align_labels, batched=True)


In [11]:
import transformers
print(transformers.__version__)


4.47.0


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./inBert-pos-results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
print(transformers.__file__)

/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [14]:
import numpy as np
from seqeval.metrics import accuracy_score, f1_score, classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2tag[label] for label in sent if label != -100]
        for sent in labels
    ]
    true_predictions = [
        [id2tag[pred] for (pred, label) in zip(sent_preds, sent_labels) if label != -100]
        for sent_preds, sent_labels in zip(predictions, labels)
    ]
    print(classification_report(true_labels,true_predictions,digits = 6))
    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions)
    }


In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    # evaluation_strategy="epoch",
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-15-616f034d73fe>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
t = trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.402500,0.388618,0.880682,0.815722
2,0.321500,0.327887,0.897146,0.837641
3,0.267400,0.313808,0.901446,0.844497


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171:

              precision    recall  f1-score   support

         ART   0.863366  0.621083  0.722452       702
     ART_NEG   0.945392  0.879365  0.911184       630
          DJ   0.620718  0.650955  0.635477      5707
          DP   0.944820  0.943110  0.943964     14361
          DV   0.664171  0.662334  0.663251      3616
         ERB   0.773030  0.755472  0.764150     10919
          ET   0.785920  0.850259  0.816824      1930
         ONJ   0.909440  0.903546  0.906483      2623
         OUN   0.724826  0.769062  0.746289     19646
         RON   0.818494  0.859591  0.838539      3326
      RON_WH   0.887814  0.816726  0.850788       562
        ROPN   0.827424  0.787083  0.806749      8717
          UM   0.925522  0.925522  0.925522      2108
           _   0.950031  0.938334  0.944147      9665

   micro avg   0.812510  0.818961  0.815722     84512
   macro avg   0.831498  0.811603  0.819701     84512
weighted avg   0.814846  0.818961  0.816425     84512



/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171:

              precision    recall  f1-score   support

         ART   0.838235  0.730769  0.780822       702
     ART_NEG   0.949593  0.926984  0.938153       630
          DJ   0.687800  0.652006  0.669425      5707
          DP   0.944739  0.954739  0.949713     14361
          DV   0.744354  0.683628  0.712700      3616
         ERB   0.809759  0.779650  0.794420     10919
          ET   0.826866  0.861140  0.843655      1930
         ONJ   0.939871  0.887915  0.913154      2623
         OUN   0.758840  0.789779  0.774000     19646
         RON   0.891487  0.859591  0.875249      3326
      RON_WH   0.934394  0.836299  0.882629       562
        ROPN   0.811672  0.829643  0.820559      8717
          UM   0.950341  0.925996  0.938011      2108
           _   0.951268  0.943197  0.947215      9665

   micro avg   0.839313  0.835976  0.837641     84512
   macro avg   0.859944  0.832953  0.845693     84512
weighted avg   0.839465  0.835976  0.837412     84512



/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171:

              precision    recall  f1-score   support

         ART   0.829819  0.784900  0.806735       702
     ART_NEG   0.941363  0.942857  0.942109       630
          DJ   0.703350  0.658490  0.680181      5707
          DP   0.950484  0.957036  0.953749     14361
          DV   0.741908  0.722622  0.732138      3616
         ERB   0.812938  0.796410  0.804589     10919
          ET   0.823558  0.873057  0.847586      1930
         ONJ   0.935152  0.901639  0.918090      2623
         OUN   0.766616  0.793189  0.779676     19646
         RON   0.883283  0.873722  0.878476      3326
      RON_WH   0.893617  0.896797  0.895204       562
        ROPN   0.835843  0.827693  0.831748      8717
          UM   0.953194  0.927419  0.940130      2108
           _   0.950327  0.946198  0.948258      9665

   micro avg   0.845163  0.843833  0.844497     84512
   macro avg   0.858675  0.850145  0.854191     84512
weighted avg   0.845115  0.843833  0.844314     84512



In [17]:
metrics = trainer.evaluate(eval_dataset=tokenized_val)
print(metrics)


              precision    recall  f1-score   support

         ART   0.829819  0.784900  0.806735       702
     ART_NEG   0.941363  0.942857  0.942109       630
          DJ   0.703350  0.658490  0.680181      5707
          DP   0.950484  0.957036  0.953749     14361
          DV   0.741908  0.722622  0.732138      3616
         ERB   0.812938  0.796410  0.804589     10919
          ET   0.823558  0.873057  0.847586      1930
         ONJ   0.935152  0.901639  0.918090      2623
         OUN   0.766616  0.793189  0.779676     19646
         RON   0.883283  0.873722  0.878476      3326
      RON_WH   0.893617  0.896797  0.895204       562
        ROPN   0.835843  0.827693  0.831748      8717
          UM   0.953194  0.927419  0.940130      2108
           _   0.950327  0.946198  0.948258      9665

   micro avg   0.845163  0.843833  0.844497     84512
   macro avg   0.858675  0.850145  0.854191     84512
weighted avg   0.845115  0.843833  0.844314     84512

{'eval_loss': 0.313808381

In [18]:
metrics

{'eval_loss': 0.3138083815574646,
 'eval_accuracy': 0.9014461654975927,
 'eval_f1': 0.8444973385201107,
 'eval_runtime': 36.789,
 'eval_samples_per_second': 135.883,
 'eval_steps_per_second': 16.989,
 'epoch': 3.0}